# Warning
This file is currently a work in progress. Expect instability, inconsistency, and ugly code.
@W

In [1]:
DATA_DIRECTORY = "../../data"

# Delete me, eventually
data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'

In [2]:
from functional import seq

In [3]:
import torch

In [4]:
# Initial object constructors
class Workspace():
    def __init__(self, tree):
        # assert tree is array
        self.tree = tree
        
    def add(address, content):
        assert isinstance(address, (str))
        assert isinstance(content, (str))
        
        def _add(tree):
            if not isinstance(tree, (list)):
                return tree
            elif tree[0] == address:
                return tree.concat([[content]])
            else:
                return map(tree, _add)
        
        return Workspace(_add(self.tree))

    def update(self, action):
        if action.what == 'INIT':
            return Workspace(['root'])
        elif action.what == 'MSG':
            return self
        elif action.what == 'ADD':
            tmp = action.content.split(' ')
            assert (len(tmp) == 2)
            address = tmp[0]
            address = tmp[1]
            return this.add(address, content)
        
    # TODO: Figure out how to flattenDeep    
    def to_words():
        return "NOT IMPLEMENTED YET"
    def to_tree():
        return this.tree
        

In [5]:
class Action():
    def __init__(self, who, what, content=False):
        self.who = who
        self.what = what
        self.content = content
        
    def to_words(self):
        return [f"{self.who}:", self.what] + (self.content.split(' ') if self.content else [])
        
    def to_string(self):
        if self.content:
            return f'{self.who}: {self.what} {self.content}'
        else:
            return f'{self.who}: {self.what}'

In [6]:
def files_to_words(filenames):
     return (seq(filenames)
                .map(lambda filename: open(filename).read().split())
                .flatten()
                .distinct()
                .sorted()
                .to_list())

In [7]:
def string_to_action(s):
    who = s[0]
    i = s.index(" ")
    line = s[i+1:]
    j = line.index(" ")
    what = line[0:j]
    content = line[j+1:]
    return Action(who, what, content)
    

In [8]:
def file_to_dialogs(filename):
    init_entry = {
        "workspace": Workspace([]),
        "action": Action(who='A', what='INIT')
    }
    
    dialog_list = []
    action_list = [init_entry]
    current_workspace = init_entry["workspace"]
    
    for line in open(filename, 'r'):
        print(line)
        break
        action_list = [init_entry]
        line = line.split('\n')
        for dialog in line:
            #line = line[:-1]
            if not dialog:
                continue
            try: 
                action_list.append({"workspace": current_workspace,
                                    "action": string_to_action(dialog)})
            except ValueError:
                print("This shouldn't happen!")
                return
        dialog_list.append(action_list)

    return dialog_list
   

In [9]:
y = file_to_dialogs(training_file)

A: ADD root cuisine



In [10]:
def file_to_dialogs(filename):
    init_entry = {
        "workspace": Workspace([]),
        "action": Action(who='A', what='INIT')
    }
    dialog_list = []
    current_workspace = init_entry["workspace"] 
    
    for dialog in open(filename).read().split('\n\n'):
        action_list = [init_entry]
        for action in dialog.split("\n"):
            action_list.append({"workspace": current_workspace,
                                "action": string_to_action(action)})
        dialog_list.append(action_list)
        
    return dialog_list
        
    

In [11]:
special_words = [
  '^', '$',            # start/end markers
  'INIT', 'WAIT',      # action types not in data
  'NO_OUTPUT',         # filler when copy or match word is not available
  'NO_USER_ACTION',    # initial value of lastUserAction
  'NO_WORKSPACE_WORD'
]

In [12]:
def load_babi_data():
    data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
    training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
    dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'
    types_and_tokens = [
            { "type": 'cuisine', "tokens": ['french', 'italian', 'british', 'spanish', 'indian'] },
            { "type": 'location', "tokens": ['rome', 'london', 'bombay', 'paris', 'madrid'] },
            { "type": 'price', "tokens": ['cheap', 'moderate', 'expensive'] },
            { "type": 'people', "tokens": ['two', 'four', 'six', 'eight'] },
        ]
    
    data = {
      "training": file_to_dialogs(training_file),
      "dev": file_to_dialogs(dev_file),
      "words": files_to_words([training_file]) + special_words,
      "types_and_tokens": types_and_tokens
    }

    return data

In [13]:
def create_one_hot(vocab_size):
    return ([[int(i == j) for i in range(vocab_size)] for j in range(vocab_size)])


In [14]:
def action_to_one_hot(action):
    return ((seq(['^'] + action.to_words() + ['$'])
                .map(lambda w: data['words'].index(w))
                .map(lambda i: one_hot[i])
                .to_list()))

In [15]:
MAX_TENSOR_SIZE = 23 #TODO: Refactor to determine automatically
def pad_tensor(tensor):
    m = MAX_TENSOR_SIZE
    a = tensor
    return a + [([0 for i in range(vocab_size)]) for j in range(m - len(a))]

In [46]:
def actions_to_one_hot(data):
    new_data = (seq(data)
                .map(lambda a: a['action'])
                .map(lambda a: action_to_one_hot(a))
                .map(pad_tensor)
                .map(torch.Tensor))
    return new_data

                #.map(action_to_one_hot))

In [47]:
def dialogs_to_one_hot(dialogs):
    out_data = []
    for dialog in dialogs:
        out = torch.stack(list(actions_to_one_hot(dialog)))
        out_data.append(out)
    return out

In [48]:
data = load_babi_data()

# Ensure it looks ok
#data['training'][100]["action"].to_string()

In [23]:
data['training'][1][-1]['action'].to_string()

'A: MSG api_call italian london eight cheap'

In [24]:
vocab_size = len(data["words"])
one_hot = create_one_hot(vocab_size)

In [50]:
# Convert our data into a list of tensors
training_data = dialogs_to_one_hot(data['training'])


# WARNING!
Everything below this cell is currently non-functional. If it compiles, I'll be rather surprised.

In [25]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [546]:
# Adapted from:
# http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

class Encoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
        self.x_size = x_size
        self.cell = nn.GRU(x_size, h_size)
        
        #self.embedding = nn.Embedding(x_size, h_size)

    # x: input
    # h: hidden state
    def forward(self, input, h):
        #embedded = self.embedding(input).view(1, 1, -1)
        output = input
        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h
    
    def init_hidden(self):
        return Variable(torch.zeros(1, 1, self.h_size)) 

In [654]:
class DecoderMock(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(DecoderMock, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
        self.x_size = x_size
        self.cell = nn.GRU(x_size, h_size)
        self.out = nn.Linear(h_size, x_size)
        self.softmax = nn.LogSoftmax()

    # x: input
    # h: hidden state
    def forward(self):
        for i in range(self.n_layers):
            output, h = self.cell(output, h)
            output = self.softmax(self.out(output[0]))
        return output, h
    
    def init_hidden(self):
        return Variable(torch.zeros(1, 1, self.h_size)) 

One day, we will be a neural network

In [547]:
hidden_size = 256
encodern = Encoder(vocab_size, hidden_size)
decodern = DecoderMock(vocab_size, hidden_size)

In [ ]:
encodern.zero_grad()
decodern.zero_grad()

encoder_hidden = encodern.init_hidden()
decoder_hidden = decodern.init_hidden()

encoder_outputs =  Variable(torch.zeros(23, encodern.h_size))

sos = data['words'].index('^')
decoder_input = Variable(torch.Tensor([[one_hot[sos]]]))

var = Variable(training_data[1].unsqueeze(1))
out, hidden = encodern(var, encoder_hidden)


In [37]:
#list(Encoder(10, 10).parameters()

In [480]:
# TODO: Implement initialization


# Whether this is a good idea is still in question
attentional_net = torch.nn.Sequential(
                    torch.nn.Linear(2, 1),
                    torch.nn.Tanh())

# Whether this is a good idea is still in question
decoder_output_net = torch.nn.Sequential(
                        torch.nn.Softmax())

class Decoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        self.n_layers = n_layers
        self.h_size = h_size
        self.cell = nn.GRU(x_size, h_size)

        self.out = nn.Linear(h_size, )
        self.attention = torch.nn.Sequential(torch.nn.Linear(2 * x_size, 1),
                                             torch.nn.Tanh())

    def forward(self, decoder_state, encoder_states):
        # Combine State
        combinedState = lambda encoder_state, z: self.attention(torch.cat(decoder_state, encoder_state), 0)

        # This is terrible and I feel dirty
        zero_states = torch.zeros(encoder_states.size()[0])

        # Create attention vector
        attention_vector = T.softmax(torch.Tensor.map_(encoder_states, zero_states,
                                                       combinedState))

        final = encoder_states * torch.transpose(attention_vector.unsqueeze(0), 0, 1).expand(encoder_states.size())

        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h